# Implementing Coverage Models in Python

## **Problem Description**

In this case study, you will implement **the maximal coverage location model** to optimize hospital placement in **Northern Baden-Württemberg**. Given a dataset containing:
1. **Towns with more than 25,000 inhabitants**, which are potential hospital locations.
3. **Travel times (in minutes) between towns** as a measure of accessibility.

<p align="center">
  <img src="../data/cities_north_bw.png" alt="Hospitals in Northern Baden-Württemberg" width="600">
</p>

As a starting point, we assume that the entire hospital network is to be revised and existing locations can be ignored entirely. 
Your objective is to **model and solve this as a Maximum Coverage Location Problem (MCLP) using SCIP**. This means that:
- A **hospital can be built in any town** from the dataset.
- A **hospital provides coverage** to the population of a town if it is within a predefined travel time of a town.
- The **goal is to place a limited number of hospitals** to **maximize the number of covered towns**.


### Mathematical Model
We define:

- $ J $ as the set of **demand locations** (towns needing coverage, indexed by $ j $).
- $ I $ as the set of **potential hospital locations** (indexed by $ i $).
- $ p $ as the **maximum number of hospitals that can be built**.
- $ a_{ij} $ as a **binary parameter**:
  - $ a_{ij} = 1 $ if hospital $ i $ can cover town $ j $ (within a given travel time threshold).
  - $ a_{ij} = 0 $ otherwise.
- $ w_{j} $ as the population living in town $ j $.
- $ y_i $ as a **binary decision variable**:
  - $ y_i = 1 $ if a hospital is built in town $ i $.
  - $ y_i = 0 $ otherwise.
- $ x_j $ as a **binary variable**:
  - $ x_j = 1 $ if town $ i $ is covered by at least one hospital.
  - $ x_j = 0 $ otherwise.

#### Objective Function
The goal is to **maximize the covered population**:
$$
\max \sum_{j \in J} w_jx_j
$$

#### Constraints
1. **Coverage Constraint**: A town is considered covered if at least one selected hospital covers it:
   $$
   x_j \leq \sum_{i \in I} a_{ij} y_i, \quad \forall j \in J
   $$
2. **Facility Limit Constraint**: The number of hospitals built cannot exceed \( p \):
   $$
   \sum_{i \in I} y_i \leq p
   $$
3. **Binary Variables**:
   $$
   x_j \in \{0,1\}, \quad y_i \in \{0,1\}, \quad \forall j \in J, \forall i \in I
   $$

In the following, we will build this model step by step!

## (1) Implement and Solve the MCLM

### ✏️ Task 1: Load Packages, Data, and Initialize the Model

Before we formulate the optimization problem, let's **load the necessary Python packages** and **initialize the SCIP model**.

#### **TODO**
1. **Import the required packages** for SCIP optimization and data handling.
2. **Load the dataset** containing:
   - The **set of towns** with more than 25,000 inhabitants (which also serve as potential hospital locations).
   - The **travel time matrix** between towns.
3. **Initialize the model parameters**, including:
   - The set of **demand locations \( J \) and potential facility locations \( I \)**.
   - The **population per town**.
   - The **coverage matrix \( a_{ij} \)**, which defines whether a town can be covered by a hospital based on travel time.
4. **Initialize the SCIP model**.




Step 1a: Load the packages 'pandas' 'numpy' and 'pysciopt'

In [34]:
# Step 1a: Load Packages
import pandas as pd
import numpy as np
from pyscipopt import Model

print("Required packages loaded.")

Required packages loaded.


Step 1b: Load Data from files and initialize parameters





In [35]:
# Step 1b: Load Data from files and initialize parameters

df_cities = pd.read_csv("../data/north_bw_cities.csv")  # Cities (J), which also serve as potential hospital locations (I)
df_travel_times = pd.read_csv("../data/north_bw_travel_times.csv", index_col=0)  # Travel time matrix

print("Data loaded successfully.")

J = set(df_cities["name"])  # Set of demand locations (also potential hospital locations)
I = J.copy()  # Since hospitals can be placed in any town

population = dict(zip(df_cities["name"], df_cities["population"]))  # Population per town

# Define coverage matrix a_ij based on a max travel time threshold (e.g., 30 min)
coverage_threshold = 30  # Maximum allowed travel time for coverage (adjustable)
a = {
    (i, j): 1 if df_travel_times.at[j, i] <= coverage_threshold else 0
    for i in I for j in J
}

# Set the number of hospitals allowed
p = 5  # Number of hospitals to be located

print("Model parameters initialized.")

Data loaded successfully.
Model parameters initialized.


Step 1c: Initialize the SCIP Model

In [36]:
 #Step 1c: Initialize the SCIP Model
model = Model("MaxCoverageHospitalLocation")

print("SCIP Model initialized.")

SCIP Model initialized.


### ✏️ Task 2: Define Decision Variables

Now that we have **loaded the data and initialized the parameters**, we will define the **decision variables** needed for our optimization model.

#### **TODO**
We define two sets of binary decision variables:

1. **Hospital location variables** \( y_i \):
   - \( y_i = 1 \) if a hospital is placed in town \( i \).
   - \( y_i = 0 \) otherwise.
   - These variables indicate where hospitals will be built.

2. **Coverage variables** \( x_j \):
   - \( x_j = 1 \) if town \( j \) is covered by at least one hospital.
   - \( x_j = 0 \) otherwise.
   - These variables indicate whether a town is covered by a hospital.

Define the binary location variable for the hospital placement via: 

`y = {i: model.addVar(vtype="B", name=f"y_{i}") for i in I}  # Hospital placement`

Copy the Python code below into a new cell and run it to define the decision variable y


In [37]:
# Definition of the decision variable y
y = {i: model.addVar(vtype="B", name=f"y_{i}") for i in I}

✏️ Can you define the decision variable `x` in a similar way? Write down your code and run the cell!

In [ ]:
# Definition of the decision variable x
# TODO: Define the decision variable x


### ✏️ Task 3: Define the Objective Function

The objective of our **Maximum Coverage Location Problem (MCLP)** is to **maximize the covered population**.

#### **Mathematical Formulation**
We maximize the **total population that is covered by at least one hospital**:
$$
\max \sum_{j \in J} w_j x_j
$$
where:
- \( w_j \) is the population of town \( j \).
- \( x_j \) is **1** if town \( j \) is covered, **0** otherwise.

✏️ We now implement this objective function in SCIP.



In [39]:
# Step 3: Define the Objective Function

# Set the objective function: Maximize the total covered population
model.setObjective(
    sum(population[j] * x[j] for j in J),  # Weighted sum of covered towns
    "maximize"  # SCIP requires specifying "maximize" or "minimize"
)

print("Objective function added to SCIP Model.")

Objective function added to SCIP Model.


### ✏️Task 4: Define Constraints

Now that we have our decision variables, we will implement the **constraints** of the model.

#### **TODO**
We will implement the following constraints:

1. **Coverage Constraint**: A town is considered **covered** if at least one hospital can serve it:
   $$
   x_j \leq \sum_{i \in I} a_{ij} y_i, \quad \forall j \in J
   $$
   - This ensures that \( x_j = 1 \) only if there is a hospital \( y_i \) that can cover town \( j \).
   - If no hospital can cover \( j \), then \( x_j = 0 \).

✏️ Copy the Python code below into a new cell and run it.

`for j in J:`

         ` model.addCons(x[j] <= sum(a[i, j] * y[i] for i in I), name=f"coverage_{j}")`


In [ ]:
# Step 3.1: Implement the Coverage Constraint

# Each town is covered if at least one hospital in range is built

# TODO: Add the coverage constraints to the model

print("Coverage constraints added to SCIP Model.")

Coverage constraints added to SCIP Model.


2. **Facility Limit Constraint**: The total number of hospitals **cannot exceed \( p \)**:
   $$
   \sum_{i \in I} y_i \leq p
   $$
   - This limits the number of hospitals that can be built.

   
✏️ Try to add the constraint yourself!

In [ ]:
# Add constraint to limit the number of selected hospital locations
# TODO: Add the constraint to limit the number of selected hospital locations

print("✅ Facility limit constraint added to SCIP Model.")

✅ Facility limit constraint added to SCIP Model.


3. **Binary Constraints**: These constraints enforce binary values:
   $$
   x_j \in \{0,1\}, \quad y_i \in \{0,1\}, \quad \forall j \in J, \forall i \in I
   $$
Thus, **no additional constraints are needed** for variable types because you already defined them as binary (B) when you initialized them. 


### Great! You implemented the model! Now, let's see if it runs

### ✏️ Task 5: Solve the Model and Log the Solver Output**

Now, we will **solve the optimization model using SCIP** and **log the solver output**.

#### **Task**
1. **Optimize the model** using SCIP.
2. **Extract and display the optimal solution**:
   - The selected hospital locations.
   - The towns that are covered.
3. **Check if an optimal solution was found** and print relevant results.

In [42]:
# Step 5: Solve the Model and Log the Solver Output

# Start the optimization process
# SCIP will now attempt to solve the problem using its branch-and-bound algorithm.
model.optimize()

# Check the solution status
# The solver returns different statuses such as "optimal", "infeasible", or "unbounded".
if model.getStatus() == "optimal":
    print("\nOptimal Solution Found!")

    # Extract the selected hospital locations
    # We check which variables y[i] are set to 1 in the optimal solution.
    selected_hospitals = [i for i in I if model.getVal(y[i]) > 0.5]
    print(f"🏥 Selected Hospital Locations: {selected_hospitals}")

    # Extract the towns that are covered
    # If x[j] is 1 in the solution, it means town j is covered by at least one hospital.
    covered_towns = [j for j in J if model.getVal(x[j]) > 0.5]
    print(f"Covered Towns: {covered_towns}")

    # Calculate the total covered population
    # We sum the population of all covered towns to measure the effectiveness of the solution.
    total_covered_population = sum(population[j] for j in covered_towns)
    print(f"👥 Total Covered Population: {total_covered_population}")

    # Display additional solver details
    print(f"\n📊 SCIP Solver Details:")
    print(f" - Objective Value (Total Covered Population): {model.getObjVal()}")
    print(f" - Fraction of the population that is covered: {round(model.getObjVal()/sum(population.values())*100,2)}%")

else:
    # Handle cases where the model is infeasible or unbounded
    print("No optimal solution found or model is infeasible. Please check your constraints and input data.")



Optimal Solution Found!
🏥 Selected Hospital Locations: ['Rheinstetten', 'Wörth am Rhein', 'Gaggenau']
Covered Towns: ['Rheinstetten', 'Herxheim bei Landau/Pfalz', 'Bad Herrenalb', 'Waldbronn', 'Karlsruhe', 'Wörth am Rhein', 'Ettlingen', 'Baden-Baden', 'Rastatt', 'Gernsbach', 'Kandel', 'Landau in der Pfalz', 'Malsch', 'Durmersheim', 'Gaggenau']
👥 Total Covered Population: 617016

📊 SCIP Solver Details:
 - Objective Value (Total Covered Population): 617016.0
 - Fraction of the population that is covered: 100.0%


## (2) Plot the Solution 

Let us take a look what this solution looks like! Simply run the following code! Try to understand what is does. 

In [43]:
import folium
# Therefore, we define ourselves a function that plots the solution on a map. We will use the folium library for this purpose.

def plot_solution(model): # Define the function!
    # Define the approximate center of northern Baden-Württemberg (around Heilbronn)
    bw_north_center = [49.2, 9.2]

    # Create a Folium map with an adjusted zoom level
    m = folium.Map(
    location=bw_north_center,
    zoom_start=9,  # Set your desired zoom level
   
    )   
    def get_coordinates(town_name):
        """
        Retrieves latitude and longitude for a given city.
        Returns (latitude, longitude) or (None, None) if the city is not found.
        """
        row = df_cities.loc[df_cities["name"] == town_name, ["latitude", "longitude"]]
        return tuple(row.iloc[0]) if not row.empty else (None, None)

    # Add cities that are covered as green markers
    for j in [j for j in J if model.getVal(x[j]) > 0.5]:
        name = j
        lan, lon = get_coordinates(j)
        folium.Marker(
            location=[lan, lon],
            popup=f"🏥 {name}",
            icon=folium.Icon(color="green", icon="home")
        ).add_to(m)

    # Add cities that are not covered as gray markers
    for j in [j for j in J if model.getVal(x[j]) <= 0.0]:
        name = j
        lan, lon = get_coordinates(j)
        folium.Marker(
            location=[lan, lon],
            popup=f"🏥 {name}",
            icon=folium.Icon(color="gray", icon="home")
        ).add_to(m)
    
    # Add hospitals as red markers
    for i in [i for i in I if model.getVal(y[i]) >= 1.0]:
        name = i
        lan, lon = get_coordinates(i)
        folium.Marker(
            location=[lan, lon],
            popup=f"🏥 {name}",
            icon=folium.Icon(color="red", icon="plus-sign")
        ).add_to(m)
    return m

# Call the function to plot the solution
m = plot_solution(model)
# Display map
m

## (3) Experiment with the Model's Parameters

Now that we have implemented the **Maximum Coverage Location Model**, it's time to **explore how changing key parameters affects the solution**.

### ✏️ Task: Adjust Parameters and Analyze Results
Modify the following parameters:
1. **The maximum number of hospitals \( p \)**:
   - Try increasing and decreasing \( p \) (e.g., \( p = 5, 10, 15 \)).
   - Observe how the selected hospital locations change.
   - Check if adding more hospitals always increases coverage significantly.

2. **The maximum coverage radius (in minutes)**:
   - Try different coverage thresholds (e.g., **20 min, 30 min, 40 min**).
   - Does a **larger coverage radius** mean fewer hospitals are needed?
   - What happens if the coverage radius is **too small**?


📌 **Your task**: Run the code below multiple times, changing \( p \) and the coverage radius each time. Observe how the solution changes.

Can you answer the following questions? 
1. What is the minimum number of hospitals you need to cover the entire population with a coverage radius of 15 minutes?
2. What is the minimum number of hospitals you need to cover the entire population with a coverage radius of 30 minutes?
3. What is the minimal coverage radius (in minutes) you can allow to cover at least 95% of the population with 10 facilities?
4. What is the minimal coverage radius (in minutes) you can allow to cover at least 90% of the population with 8 facilities?


Critically reflect on what areas of the map are covered and which ones are not. What does this tell you about the fairness of the solution in the maximum coverage location model?

In [ ]:

# TODO: Modify the parameters and run the model to see how the solution changes.
p = 1000  # Maximum number of hospitals (Try changing this)
coverage_threshold = 1000  # Maximum allowed travel time for coverage (adjustable)

a = {
    (i, j): 1 if df_travel_times.at[j, i] <= coverage_threshold else 0
    for i in I for j in J
}

# Below, the model is re-implemented to avoid and potential changes from previous consoles.
model = Model("MaxCoverageHospitalLocation")
y = {i: model.addVar(vtype="B", name=f"y_{i}") for i in I}  # Hospital placement
x = {j: model.addVar(vtype="B", name=f"x_{j}") for j in J}  # Town coverage
for j in J: model.addCons(x[j] <= sum(a[i, j] * y[i] for i in I), name=f"coverage_{j}")
model.addCons(sum(y[i] for i in I) <= p, name="facility_limit")
model.setObjective(sum(population[j] * x[j] for j in J), "maximize")
model.optimize()

# ✅ Step 10: Extract results
if model.getStatus() == "optimal":
    # Calculate total covered population
    covered_towns = [j for j in J if model.getVal(x[j]) > 0.5]
    total_covered_population = sum(population[j] for j in covered_towns)
    total_population = sum(population.values())
    coverage_percentage = (total_covered_population / total_population) * 100
    print("\n✅ Optimal Solution Found!")
    print(f"📊 Percentage of Population Covered: {coverage_percentage:.2f}%")
    m = plot_solution(model)
       

else:
    print("⚠️ No optimal solution found. Adjust parameters and try again.")
#   Display map
m


✅ Optimal Solution Found!
📊 Percentage of Population Covered: 89.09%
